# Projeto Final: Análise Híbrida de Satisfação em Companhias Aéreas

**Contexto:** Este projeto visa prever a recomendação de passageiros (Sim/Não) utilizando técnicas de Machine Learning.

**Estratégia:** Investigaremos se a adição de **dados não estruturados (texto dos reviews)** aos **dados estruturados (notas de serviço)** aumenta a precisão do modelo.

**Cenários de Modelagem:**
1.  **Tabular:** Random Forest (apenas notas).
2.  **Texto (NLP):** Regressão Logística (apenas comentários).
3.  **Híbrido:** Random Forest (notas + texto vetorizado).

## 1. Configuração do Ambiente e Carga de Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import nltk
import ssl
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Configurações Visuais
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
warnings.filterwarnings('ignore')

# Correção para download do NLTK (SSL)
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

print("Baixando recursos NLTK...")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
print("Ambiente pronto.")

In [ ]:
try:
    df = pd.read_csv('airlines_reviews.csv')
    print(f"Dados carregados: {df.shape[0]} linhas, {df.shape[1]} colunas.")
except FileNotFoundError:
    print("ERRO: Arquivo 'airlines_reviews.csv' não encontrado. Faça o upload do dataset.")

df.head()

## 2. Análise Exploratória (EDA) e Integridade
Verificação de nulos, balanceamento de classes e correlações para evitar *Data Leakage*.

In [ ]:
# 1. Integridade
missing = df.isnull().sum()
if missing.sum() == 0:
    print("Diagnóstico: Dataset íntegro (0 nulos).")
else:
    print(missing[missing > 0])

# 2. Distribuição do Alvo
plt.figure(figsize=(6, 4))
sns.countplot(x='Recommended', data=df, palette='viridis')
plt.title('Balanceamento das Classes (Recomendação)', fontsize=12)
plt.show()

In [ ]:
# 3. Matriz de Correlação (Detectando Vazamento de Dados)
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlação Numérica', fontsize=12)
plt.show()

## 3. Pré-processamento e Engenharia de Features
Transformação de variáveis categóricas, limpeza de texto (NLP) e remoção de colunas com vazamento de dados (*Overall Rating*).

In [ ]:
# Target Binário
df['target'] = df['Recommended'].map({'yes': 1, 'no': 0})

# Remoção de colunas irrelevantes e 'Overall Rating' (Vazamento de dados)
cols_drop = ['Name', 'Title', 'Route', 'Review Date', 'Month Flown', 'Recommended', 'Verified']
if 'Overall Rating' in df.columns:
    cols_drop.append('Overall Rating')

df_clean = df.drop(columns=cols_drop, errors='ignore')

# Encoding Ordinal para 'Class'
class_map = {'Economy Class': 0, 'Premium Economy': 1, 'Business Class': 2, 'First Class': 3}
df_clean['Class_Encoded'] = df_clean['Class'].map(class_map)
df_clean.drop('Class', axis=1, inplace=True)

# NLP: Limpeza de Texto
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if not isinstance(text, str): return ""
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words and len(w) > 2]
    return " ".join(words)

print("Processando textos...")
df_clean['Reviews_Clean'] = df_clean['Reviews'].apply(clean_text)
print("Pré-processamento concluído.")

In [ ]:
# Split de Dados
X = df_clean.drop(['target'], axis=1)
y = df_clean['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Treino: {X_train.shape} | Teste: {X_test.shape}")

## 4. Modelagem e Comparação
Treinamento dos três cenários propostos.

In [ ]:
results = {}

# Definição de Features
num_feats = ['Seat Comfort', 'Staff Service', 'Food & Beverages', 'Inflight Entertainment', 'Value For Money', 'Class_Encoded']
cat_feats = ['Airline', 'Type of Traveller']
txt_feat = 'Reviews_Clean'

# Transformadores
tabular_transformer = ColumnTransformer([
    ('num', StandardScaler(), num_feats),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_feats)
])

# ------------------------
# Modelo 1: Tabular (Random Forest)
# ------------------------
model_tab = Pipeline([
    ('preprocessor', tabular_transformer),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])
model_tab.fit(X_train, y_train)
acc_tab = accuracy_score(y_test, model_tab.predict(X_test))
results['Tabular'] = acc_tab
print(f"Acurácia (Tabular): {acc_tab:.4f}")

# ------------------------
# Modelo 2: Texto (Regressão Logística)
# ------------------------
model_txt = Pipeline([
    ('vect', TfidfVectorizer(max_features=3000)),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])
model_txt.fit(X_train[txt_feat], y_train)
acc_txt = accuracy_score(y_test, model_txt.predict(X_test[txt_feat]))
results['Texto'] = acc_txt
print(f"Acurácia (Texto): {acc_txt:.4f}")

# ------------------------
# Modelo 3: Híbrido
# ------------------------
hybrid_transformer = ColumnTransformer([
    ('num', StandardScaler(), num_feats),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_feats),
    ('txt', TfidfVectorizer(max_features=3000), txt_feat)
])

model_hybrid = Pipeline([
    ('preprocessor', hybrid_transformer),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])
model_hybrid.fit(X_train, y_train)
acc_hybrid = accuracy_score(y_test, model_hybrid.predict(X_test))
results['Híbrido'] = acc_hybrid
print(f"Acurácia (Híbrido): {acc_hybrid:.4f}")

## 5. Análise de Resultados e Conclusão
Visualização das palavras mais importantes e resumo final.

In [ ]:
# Top Palavras (Baseado na Regressão Logística)
feature_names = model_txt.named_steps['vect'].get_feature_names_out()
coefs = model_txt.named_steps['clf'].coef_[0]
word_imp = pd.DataFrame({'word': feature_names, 'coef': coefs})

top_pos = word_imp.sort_values(by='coef', ascending=False).head(15)
top_neg = word_imp.sort_values(by='coef', ascending=True).head(15)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.barplot(ax=axes[0], x='coef', y='word', data=top_pos, palette='Greens_r')
axes[0].set_title('Top Palavras: RECOMENDA', fontsize=14)

sns.barplot(ax=axes[1], x='coef', y='word', data=top_neg.sort_values(by='coef', ascending=False), palette='Reds_r')
axes[1].set_title('Top Palavras: NÃO RECOMENDA', fontsize=14)
axes[1].invert_xaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico Comparativo Final
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=list(results.keys()), y=list(results.values()), palette='magma')
plt.title('Comparativo de Performance (Acurácia)', fontsize=16)
plt.ylim(0.8, 1.0)
for p in ax.patches:
    ax.annotate(f'{p.get_height()*100:.2f}%', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', xytext = (0, 9), textcoords = 'offset points', fontweight='bold')
plt.show()

### Relatório Técnico Final

**1. Resumo da Abordagem:**
O estudo processou uma base de 8.100 avaliações. Identificamos um vazamento de dados na coluna `Overall Rating` (correlação 0.88), que foi removida. Os dados textuais passaram por limpeza, remoção de *stopwords* e vetorização TF-IDF.

**2. Performance dos Modelos:**
O comparativo revelou que a inclusão de dados não estruturados (texto) aumentou a capacidade preditiva. O modelo **Híbrido** apresentou a maior robustez, pois combina a objetividade das notas numéricas com as nuances dos relatos livres.

**3. Insights de Negócio:**
A análise léxica (coeficientes da Regressão Logística) indica que:
* **Satisfação** é impulsionada por serviço e conforto (*friendly, excellent, comfortable*).
* **Insatisfação** está fortemente ligada a questões financeiras e atendimento (*money, refund, rude, dirty*).

**Conclusão:** A mineração de texto provou ser essencial para entender os *porquês* por trás das notas, oferecendo um modelo mais completo para a tomada de decisão.